In [1]:
import json
import pandas as pd
import numpy as np
import pandas as pd
import os
import seaborn as sns
import ast
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from collections import Counter

def metrics(actual, model):
    filtered_actual = []
    filtered_model = []

    for i, j in zip(actual, model):
        if j != -1:
            filtered_actual.append(i)
            filtered_model.append(j)
    
    total_responses = len(actual)
    valid_responses = len(filtered_actual)
    valid_percentage = (valid_responses / total_responses)
    
    f1_class_op = f1_score(filtered_actual, filtered_model, labels=[0], average='macro')
    f1_class_su = f1_score(filtered_actual, filtered_model, labels=[1], average='macro')
    f1_class_nu = f1_score(filtered_actual, filtered_model, labels=[2], average='macro')
    weighted_f1 = f1_score(filtered_actual, filtered_model, average='weighted')
    accuracy = accuracy_score(filtered_actual, filtered_model)
    
    
    return {
        'Valid Resps': valid_percentage,
        'F1 Opp': f1_class_op,
        'F1 Sup': f1_class_su,
        'F1 Neu': f1_class_nu,
        'W-F1': weighted_f1,
        'ACC': accuracy,
    }

In [2]:
mo = pd.read_csv('./mo-l3170.csv')
mo = mo.drop({"Unnamed: 0", 'actual', 'meta-llama/Meta-Llama-3.1-70B-Instruct-Probs'}, axis = 1)
mo = mo.rename(columns={'meta-llama/Meta-Llama-3.1-70B-Instruct': 'pred-mo'})


la = pd.read_csv('./test_total/Per_model_output/l3170i-t7.csv')
la = la.drop({"Unnamed: 0"}, axis = 1)
la = la.rename(columns={'meta-llama/Meta-Llama-3.1-70B-Instruct': 'pred'})

lp = pd.read_csv('./test_total/instruct_class_proba/l3170i-t7.csv')
lp = lp.drop({"Unnamed: 0", 'actual', 'meta-llama/Meta-Llama-3.1-70B-Instruct'}, axis = 1)
lp = lp.rename(columns={'meta-llama/Meta-Llama-3.1-70B-Instruct-Probs': 'probs'})


merged_lp_mo = pd.merge(lp, mo, on='id', how='inner')
merged = pd.merge(merged_lp_mo, la, on='id', how='inner')

In [3]:
merged.head(1)

,id,probs,pred-mo,actual,pred
0,6689,"[0.07671631, 0.824778, 0.09850569]",1,0,1


In [9]:
final_pred = []
final_test = []
for index, row in merged.iterrows():
    final_test.append(row['actual'])
    if ast.literal_eval(row['probs'])[row['pred']] < 0.6 or (row['pred-mo'] == 1):
        final_pred.append(row['pred-mo'])
    else:
        final_pred.append(row['pred'])

In [10]:
metrics(final_test, merged['pred']), metrics(final_test, final_pred)

({'Valid Resps': 0.930035335689046,
  'F1 Opp': 0.7578040904198062,
  'F1 Sup': 0.6818851251840943,
  'F1 Neu': 0.8349609375000001,
  'W-F1': 0.7642670305360079,
  'ACC': 0.7682370820668692},
 {'Valid Resps': 0.9763250883392226,
  'F1 Opp': 0.7461250668091929,
  'F1 Sup': 0.7146226415094339,
  'F1 Neu': 0.8392036753445636,
  'W-F1': 0.7699751320177106,
  'ACC': 0.7694534925805284})

/var/folders/5_/ct_h64s936q91wl1qyv28l040000gn/T/ipykernel_16495/879034021.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = r2.to_latex(index=False)
